In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import tensorflow as tf
#tf.set_random_seed(42)

import keras
import pandas as pd
import os
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, BatchNormalization, Dropout, GlobalMaxPooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
path = '/content/drive/My Drive/ColabNotebooks/CNN/ASSIGNMENT/'
os.chdir(os.path.dirname(os.path.abspath(path)))

print(os.getcwd())

/content/drive/My Drive/ColabNotebooks/CNN


In [ ]:
!ls -l ASSIGNMENT

total 257
-rw------- 1 root root  41811 Sep  7 06:03 'Computer Vision with CNN - Project 1.ipynb'
-rw------- 1 root root 212574 Sep  6 12:45 'Computer Vision with CNN_R7_Project1_Plant seedling.ipynb'
drwx------ 2 root root   4096 Aug 14 09:46  test
drwx------ 2 root root   4096 Aug 14 09:46  train


In [ ]:
train_path = 'ASSIGNMENT/train'
test_path = 'ASSIGNMENT/test'
os.listdir(train_path)

['Scentless Mayweed',
 'Loose Silky-bent',
 'Cleavers',
 'Shepherds Purse',
 'Common Chickweed',
 'Charlock',
 'Fat Hen',
 'Common wheat',
 'Maize',
 'Black-grass',
 'Small-flowered Cranesbill',
 'Sugar beet']

In [ ]:
#Define some parameters
img_size = 60
img_depth = 3

In [ ]:
#ImageDataGenerator declaration with 20% data as test (80% for training)
img_generator= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=180,
                                                               width_shift_range=0.2,
                                                               height_shift_range=0.2,
                                                               horizontal_flip=True,
                                                               vertical_flip = True,
                                                               validation_split=0.2)

In [ ]:
#Build training generator. 
train_generator = img_generator.flow_from_directory(train_path,
                                                    target_size=(img_size, img_size),
                                                    color_mode="rgb",
                                                    subset='training',
                                                    class_mode="categorical",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    seed=42)


Found 3260 images belonging to 12 classes.


In [ ]:
#Build validation generator
validation_generator = img_generator.flow_from_directory(train_path,
                                                   target_size=(img_size, img_size),   
                                                   color_mode="rgb",                                                
                                                   subset='validation',
                                                   class_mode="categorical",
                                                   batch_size=64,
                                                   shuffle=True,
                                                   seed=42)

Found 811 images belonging to 12 classes.


In [ ]:
test_generator = img_generator.flow_from_directory(test_path,
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode=None,
    batch_size=1    
)

Found 798 images belonging to 1 classes.


In [ ]:
X, y = next(train_generator)

print('Input features shape', X.shape)
print('Actual labels shape', y.shape)

Input features shape (64, 60, 60, 3)
Actual labels shape (64, 12)


In [ ]:
y[15]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()

#normalize data
model.add(BatchNormalization(input_shape=(img_size,img_size,3,)))

#Add Conv Layer
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))


model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))

model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


#Add Output Layer
model.add(tf.keras.layers.Dense(12, activation='softmax'))

In [ ]:
#To use adam optimizer for learning weights with learning rate = 0.001
optimizer = Adam(lr=0.001)

In [ ]:
#Specify Loass and Optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10)

In [ ]:
model_checkpoint =  ModelCheckpoint('Plant_seeding_{epoch:02d}_loss{val_loss:.4f}.h5',
                                                           monitor='val_loss',
                                                           verbose=1,
                                                           save_best_only=True,
                                                           save_weights_only=True,
                                                           mode='auto',
                                                           period=1)

In [ ]:
#Model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 60, 60, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 58, 58, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 58, 58, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 28, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 28, 28, 64)        2

In [ ]:
model.fit_generator(train_generator, 
                          epochs=200,
                          steps_per_epoch= 3260//64,  #Number of training images//batch_size
                          validation_data=validation_generator,
                          validation_steps = 811//64
                    ,callbacks=[model_checkpoint]
                    ) #Number of test images//batch_size

Epoch 1/200
50/50 [==============================] - ETA: 0s - loss: 2.7244 - accuracy: 0.2635
Epoch 00001: val_loss improved from inf to 8.43239, saving model to Plant_seeding_01_loss8.4324.h5
50/50 [==============================] - 48s 965ms/step - loss: 2.7244 - accuracy: 0.2635 - val_loss: 8.4324 - val_accuracy: 0.1589
Epoch 2/200
50/50 [==============================] - ETA: 0s - loss: 1.9646 - accuracy: 0.4058
Epoch 00002: val_loss improved from 8.43239 to 4.60355, saving model to Plant_seeding_02_loss4.6036.h5
50/50 [==============================] - 47s 947ms/step - loss: 1.9646 - accuracy: 0.4058 - val_loss: 4.6036 - val_accuracy: 0.1654
Epoch 3/200
50/50 [==============================] - ETA: 0s - loss: 1.5989 - accuracy: 0.4822
Epoch 00003: val_loss did not improve from 4.60355
50/50 [==============================] - 48s 954ms/step - loss: 1.5989 - accuracy: 0.4822 - val_loss: 5.3560 - val_accuracy: 0.1914
Epoch 4/200
50/50 [==============================] - ETA: 0s - los

In [ ]:
# evaluate model
model.evaluate_generator(validation_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.2891152501106262, 0.9161528944969177]

In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
798/798 [==============================] - 252s 315ms/step


In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)